In [2]:
2do Parcial Inteligencia Artificial 1 (SIS420)

Univ. Lino Fernando Villca Jaita (Ing. en Ciencias De La Computación)

import numpy as np

class Board():
    def __init__(self):
        self.state = np.zeros((6, 7))  # Tablero de 6 filas por 7 columnas

    def valid_moves(self):
        return [(i, j) for j in range(7) for i in range(6) if self.state[0, j] == 0]

    def update(self, symbol, row, col):
        for i in range(5, -1, -1):
            if self.state[i, col] == 0:
                self.state[i, col] = symbol
                return
        raise ValueError("Movimiento ilegal!")

    def is_game_over(self):
        # Comprobar filas, columnas y diagonales
        for row in range(6):
            for col in range(7):
                if col <= 3 and np.all(self.state[row, col:col + 4] == 1):
                    return 1
                if col <= 3 and np.all(self.state[row, col:col + 4] == -1):
                    return -1
                if row <= 2 and np.all(self.state[row:row + 4, col] == 1):
                    return 1
                if row <= 2 and np.all(self.state[row:row + 4, col] == -1):
                    return -1
                if col <= 3 and row <= 2 and np.all([self.state[row + i, col + i] == 1 for i in range(4)]):
                    return 1
                if col <= 3 and row <= 2 and np.all([self.state[row + i, col + i] == -1 for i in range(4)]):
                    return -1
                if col >= 3 and row <= 2 and np.all([self.state[row + i, col - i] == 1 for i in range(4)]):
                    return 1
                if col >= 3 and row <= 2 and np.all([self.state[row + i, col - i] == -1 for i in range(4)]):
                    return -1
        # Empate
        if len(self.valid_moves()) == 0:
            return 0
        # Seguir jugando
        return None

    def reset(self):
        self.state = np.zeros((6, 7))



In [3]:
from tqdm import tqdm

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins

    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:  # Empate
            for player in self.players:
                player.reward(0.5)
        else:  # Le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

In [4]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}  # Tabla con pares estado -> valor
        self.alpha = alpha        # Learning rate
        self.positions = []       # Guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp  # Probabilidad de explorar

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # Exploracion
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # Vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # Explotacion
        # Vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(6*7))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(6*7)))

    def reward(self, reward):
        # Al final de la partida (cuando recibimos la recompensa)
        # Iteramos por todos los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]


In [ ]:
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(300000)


  3%|▎         | 9025/300000 [25:10<7:26:33, 10.86it/s]

In [ ]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla


In [ ]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)